In [1]:
import geopandas as gpd
import pandas as pd
import warnings
# Apagar todos los warnings
warnings.filterwarnings("ignore")

load_block_cols = ['MANZENT_I', 'Number_res', 'Number_hou', 'ID_PLACA', 'geometry']

In [38]:
# Actuales
gdf = gpd.read_file('/app/data/manzanas_hab/Manzanas base')
# [load_block_cols]
gdf['pers'] = gdf[['Number_res', 'TOTAL_PERS']].max(axis=1)
traslate = {
    'MANZENT_I': 'block_id',
    'pers': 'q_residents',
}
gdf.rename(columns=traslate, inplace=True)
gdf['ID_PLACA'].fillna(0,inplace=True)
save_cols = ['block_id', 'q_residents', 'geometry']
gdf = gdf[save_cols]
def fix_ids(gdf):
    mask = gdf['block_id'].isna()
    not_id = gdf.loc[mask, :]
    yes_id = gdf.loc[~mask, :]
    not_id.loc[:, 'block_id'] = ['2024']*len(not_id) + not_id.index.astype(str)
    gdf = pd.concat([yes_id, not_id])
    return gdf
def fix_duplicate_ids(gdf):
    duplicate_mask = gdf['block_id'].duplicated(keep=False)
    gdf.loc[duplicate_mask, 'block_id'] = gdf.loc[duplicate_mask, 'block_id'].astype(str) + gdf[duplicate_mask].index.astype(str)
    return gdf
gdf = fix_ids(gdf)
gdf = fix_duplicate_ids(gdf)
gdf = gpd.GeoDataFrame(data=gdf.drop(columns=['geometry']), geometry=gdf['geometry'])
gdf.to_parquet('/app/assets/blocks/actual.parquet')

In [39]:
actual = gdf.copy()

In [4]:
gdf = gpd.read_file('/app/data/manzanas_hab/Manzanas futuro')
# [load_block_cols]
traslate = {
    'MANZENT_I': 'block_id',
    'total_p_34': 'q_residents',
}
gdf.rename(columns=traslate, inplace=True)
gdf['ID_PLACA'].fillna(0,inplace=True)

def fix_ids(gdf):
    mask = gdf['block_id'].isna()
    not_id = gdf.loc[mask, :]
    yes_id = gdf.loc[~mask, :]
    not_id.loc[:, 'block_id'] = ['2024']*len(not_id) + not_id.index.astype(str)
    gdf = pd.concat([yes_id, not_id])
    return gdf

gdf = fix_ids(gdf)
gdf = gdf[save_cols]
gdf.to_parquet('/app/assets/blocks/future.parquet')

In [5]:
future = gdf.copy()

In [6]:
future['block_id'].isin(actual['block_id']).sum()

162

In [22]:
# para exportarlo como unidad
len(actual['block_id'].unique())

200

In [41]:
duplicate_mask = actual['block_id'].duplicated(keep=False)
actual[duplicate_mask]

,block_id,q_residents,geometry


In [48]:
mask = (actual['block_id'].str.len() > 13)
actual[mask]

,block_id,q_residents,geometry
32,810105100202332,0.0,"POLYGON Z ((672758.284 5922593.473 0.000, 6727..."
149,8101051002023149,0.0,"POLYGON Z ((672810.680 5922502.510 0.000, 6728..."


In [52]:
actual.drop(columns=['q_residents']).to_parquet('/app/assets/groupby/block/actual.parquet')
future.drop(columns=['q_residents']).to_parquet('/app/assets/groupby/block/future.parquet')